_________________

# Imports

Import all relevant dependencies
_________________

In [ ]:
import torch
from utils import plot_digits, plot_original_vs_encoded

from models.autoencoder.autoencoder import  Autoencoder
from models.ebm.energy_net import EnergyNet

import matplotlib.pyplot as plt


_________________

# Load Autoencoder and EBM
The best parameters are saved with the prefix 'best' in the checkpoints folder
_________________

In [ ]:
# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load Autoencoder
autoencoder = Autoencoder(input_dim=28*28, hidden_dim=1024, encoded_dim=4)
autoencoder.to(device)

# Load pretrained best checkpoint
best_autoencoder_ckpt_path = './saved_ckpts/autoencoder/autoencoder_best.pth'
autoencoder.load_pretrained_model(model_ckpt_path=best_autoencoder_ckpt_path)

In [ ]:
# Load EBM
energy_model = EnergyNet(in_dim=autoencoder.encoded_dim, autoencoder_ckpt_path=autoencoder.ckpt_path ,hid_dim=8)
energy_model.to(device)

# Load pretrained best checkpoint
best_ebm_ckpt_path = './saved_ckpts/ebm/ebm_best.pth'
energy_model.load_pretrained_model(model_ckpt_path=best_ebm_ckpt_path)

_________________________
# Autoencoder : Inference
Lets first see how the autoencoder performs
_________________________


In [ ]:
# Run this after running the Dataset section of train.ipynb
X_test   = torch.load('./data/MNIST_binary/X_test.pt').to(torch.float32)

In [ ]:
# Pass images through autoencoder
for idx in range(5):
    plot_original_vs_encoded(x=X_test[idx], model=autoencoder)

______________________________

# EBM : Inference
Once the autoencoder is ready, we move onto see the results of trained EBM
______________________________



In [ ]:
# Generate images from energy model
generated_images  = energy_model.generate_images(autoencoder=autoencoder, num_images=25, device=device)

In [ ]:
self = energy_model
from models.ebm.energy_net_utils import langevin_MCMC
X, X_samples = langevin_MCMC(f_theta=self, input_dim=autoencoder.encoded_dim,  
                        batch_size=5, num_steps=10000, eps=.001, interval_samples=True)

# generated_data = autoencoder.decoder.predict(x_sample.to(device))
# generated_data = generated_data.detach().to('cpu')
# generated_data = generated_data.reshape(-1,28,28)

In [ ]:
generated_images_sequence = []

for x_sample in  X_samples:

    generated_data = autoencoder.decoder.predict(x_sample.to(device))
    generated_data = generated_data.detach().to('cpu')
    generated_data = generated_data.reshape(-1,28,28)
    
    generated_images_sequence.append(generated_data)
    

In [ ]:
# Visualise the generated images
for idx in range(15):
    plot_digits(X=generated_images_sequence[idx], Y=None, n=5, random=False)
    # Clear the current figure
    plt.show()
    plt.clf()

In [ ]:
from models.ebm.energy_net_utils import langevin_MCMC


In [ ]:
# Visualise the generated images
plot_digits(X=generated_images, Y=None, n=25)

_______________________

# THANK YOU
_______________________


In [ ]:
from models.ebm.energy_net_utils import langevin_MCMC

In [ ]:
energy_model

In [ ]:
X, X_samples =  langevin_MCMC(f_theta=energy_model, input_dim=autoencoder.encoded_dim,  
                              batch_size=5, num_steps=10000, eps=.001, 
                              interval_samples=True)

# generated_data = autoencoder.decoder.predict(x_sample.to(device))
# generated_data = generated_data.detach().to('cpu')
# generated_data = generated_data.reshape(-1,28,28)

In [ ]:
X_samples[3]

In [ ]:
X

In [ ]:
generated_data = autoencoder.decoder.predict(X_samples[1].to(device))
generated_data = generated_data.detach().to('cpu')
generated_data = generated_data.reshape(-1,28,28)

In [ ]:
generated_data

In [ ]:
plot_digits(X=generated_images, Y=None, n=5)